In [16]:
#Author Tanmay

import pandas as pd
from gensim import corpora, models
import gensim
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import CoherenceModel
import string
from wordcloud import WordCloud

In [17]:
# 1. Data Loading
data = pd.read_csv('foodstamp_submissions_allyears.csv')

#2 Preprocessing

filtered_data = data[
    ~data['selftext'].str.contains('\[deleted\]', case=False, na=False) &
    ~data['selftext'].str.contains('\[removed\]', case=False, na=False)
]

# 3. Data Preprocessing
# Handle NaN values and ensure the data type is string
filtered_data['selftext'] = filtered_data['selftext'].fillna('')
#Removing any links
filtered_data['selftext'] = filtered_data['selftext'].str.replace(r'http\S+', '', regex=True)
# Remove punctuation, numbers, and special characters
filtered_data['clean_selftext'] = filtered_data['selftext'].map(lambda x: re.sub('[,\.!?]', '', x))
filtered_data['clean_selftext'] = filtered_data['clean_selftext'].map(lambda x: re.sub('\d+', '', x))
# Convert to lowercase
filtered_data['clean_selftext'] = filtered_data['clean_selftext'].map(lambda x: x.lower())


In [18]:
#Creating a list from the dataframe as BERT expects list of docs as inputs
docs = filtered_data['clean_selftext'].tolist()

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a document
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Apply the function to your documents
docs_no_stopwords = [remove_stopwords(doc) for doc in docs]


In [19]:
from bertopic import BERTopic

# Create a BERTopic instance
topic_model = BERTopic()

# Fit the model and transform your documents into topics
topics, probs = topic_model.fit_transform(docs_no_stopwords)


In [23]:
# Get an overview of the topics
topic_model.get_topic_info()

# Retrieve individual topics
#topic_model.get_topic(0)  # Replace 0 with the topic number you're interested in


[('xmas', 1.635204719456826),
 ('gtgt', 1.635204719456826),
 ('election', 1.4782631603369536),
 ('cut', 0.9113165770548401),
 ('see', 0.7300504513342656),
 ('apply', 0.7280645569982707),
 ('one', 0.6404719649211392),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [21]:
# Visualize topics
topic_model.visualize_topics()


/Users/tanmaybhardwaj/anaconda3/lib/python3.10/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.



TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

In [22]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,hi im wondering can you be married and still g...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
1,not sure if this is only pa but i'm sure it co...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
2,today i was informed by my social worker that ...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
3,(i'm in alabama)\n\nso i filled out the online...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
4,hello\n\ni am in new york\n\ni applied on apri...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
...,...,...,...,...,...,...,...,...
6337,i recently got approved for early learning coa...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
6338,hello\n\ni really hate to bother asking here b...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
6339,a friend of mine receives ssi payments for her...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,0.936643,False
6340,i looked outside my door and there was a card ...,0,0_im_get_food_benefits,"[im, get, food, benefits, month, income, card,...",[receiving food stamps california months (i th...,im - get - food - benefits - month - income - ...,1.000000,False
